In [ ]:
%matplotlib inline
#這是jupyter notebook的magic word˙

import matplotlib
import matplotlib.pyplot as plt
from IPython import display

In [ ]:
import os
#判斷是否在jupyter notebook上
def is_in_ipython():
    "Is the code running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))

    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'jupyter' in program_name or  # jupyter
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False


#判斷是否在colab上
def is_in_colab():
    if not is_in_ipython(): return False
    try:
        from google import colab
        return True
    except: return False

#判斷是否在kaggke_kernal上
def is_in_kaggle_kernal():
    if 'kaggle' in os.environ['PYTHONPATH']:
        return True
    else:
        return False

if is_in_colab():
    from google.colab import drive
    drive.mount('/content/gdrive')

os.environ['TRIDENT_BACKEND'] = 'pytorch'
kaggle_kernal=None
if is_in_kaggle_kernal():
    os.environ['TRIDENT_HOME'] = './trident'
    
elif is_in_colab():
    os.environ['TRIDENT_HOME'] = '/content/gdrive/My Drive/trident'
  


In [ ]:
#為確保安裝最新版 

!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.4-py3-none-any.whl --upgrade

import re
import pandas
import json
import copy

import numpy as np
#調用trident api
import subprocess
import random
from tqdm import tqdm
import scipy
import time
import glob
import pandas as pd
from shutil import copyfile
from torch import nn

In [ ]:
import trident as T
from trident import *

palette = [(255, 192, 0), (0, 192, 255), (128, 0, 255), (192,255,64), (255, 0, 128), (64,255,128)]

In [ ]:
# Read in data files
BASE_DIR = '../input/nfl-health-and-safety-helmet-assignment'

# 標籤與提交範例
train_labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
ss = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

# 球員追蹤數據
train_player_tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
test_player_tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')

# Baseline helmet detection labels
train_baseline_helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
test_baseline_helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

# 額外的圖像標註
img_labels = pd.read_csv(f'{BASE_DIR}/image_labels.csv')





img_labels

In [ ]:
extra_images=glob.glob('../input/nfl-health-and-safety-helmet-assignment/images/*.jpg')
print(len(extra_images))

img_labels_frequency=img_labels['label'].value_counts()
print(img_labels_frequency)

label2index = {
    'Helmet': 0,
    'Helmet-Blurred': 1,
    'Helmet-Difficult': 2,
    'Helmet-Sideline': 3,
    'Helmet-Partial': 4
}

folder,filename,ext=split_path(extra_images[0])
sample_data=img_labels[img_labels['image']==filename+ext]
sample_data


In [ ]:
img=image2array(extra_images[0])
pillow_img=array2image(img)

for index, item in sample_data.iterrows():
    box=[int(item['left']),int(item['top']),int(item['left'])+int(item['width']),int(item['top'])+int(item['height'])]
    pillow_img =plot_bbox(box, pillow_img, palette[label2index[item['label']]] ,item['label'], line_thickness=2)

display.display(pillow_img)

In [ ]:
bboxes_dataset_dict=unpickle('../input/nfl-health-safety-baseline/bboxes_dataset_dict.pkl')
video_bboxes_dataset_dict=unpickle('../input/nfl-health-safety-baseline/video_bboxes_dataset_dict.pkl')
print(len(bboxes_dataset_dict))
print(len(video_bboxes_dataset_dict))

In [ ]:
from trident.models import ssd,rfbnet

#pretrainedmodel=rfbnet.RfbNet(pretrianed=True)
#pretrainedmodel.summary()

rfbmodel=rfbnet.RfbNet(pretrianed=True, num_classes=2, num_regressors=4)
print(rfbmodel.model.priors)
rfbmodel.summary()

In [ ]:

def intersect(box_a, box_b):
    """ We Resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    A = box_a.size(0)
    B = box_b.size(0)
    max_xy = torch.min(box_a[:, 2:4].unsqueeze(1).expand(A, B, 2), box_b[:, 2:4].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, 0:2].unsqueeze(1).expand(A, B, 2), box_b[:, 0:2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]


def jaccard(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.  Here we operate on
    ground truth boxes and default boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2] - box_a[:, 0]) * (box_a[:, 3] - box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2] - box_b[:, 0]) * (box_b[:, 3] - box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter,union  # [A,B]



def match(truths, priors, variances, labels, keypoints=None,threshold=0.3):
    """Match each prior box with the ground truth box of the highest jaccard
    overlap, encode the bounding boxes, then return the matched indices
    corresponding to both confidence and location preds.
    Args:

        truths: (tensor) Ground truth boxes, Shape: [num_obj, num_priors].
        priors: (tensor) Prior boxes from priorbox layers, Shape: [n_priors,4].
        variances: (tensor) Variances corresponding to each prior coord,
            Shape: [num_priors, 4].
        labels: (tensor) All the class labels for the image, Shape: [num_obj].
        loc_t: (tensor) Tensor to be filled w/ endcoded location targets.
        conf_t: (tensor) Tensor to be filled w/ matched indices for conf preds.
        idx: (int) current batch index
        threshold: (float) The overlap threshold used when mathing boxes.

    Return:
        The matched indices corresponding to 1)location and 2)confidence preds.
    """
    # jaccard index
    priors2 = priors.clone()
    num_priors = len(priors)
    overlaps = jaccard(truths, xywh2xyxy(priors.clone()))
    # (Bipartite Matching)
    # [1,num_objects] best prior for each ground truth
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)

    # ignore hard gt
    valid_gt_idx = best_prior_overlap[:, 0] >= 0.2
    best_prior_idx_filter = best_prior_idx[valid_gt_idx, :]
    if best_prior_idx_filter.shape[0] <= 0:
        return np.zeros((num_priors, 4)).astype(np.float32), np.zeros((num_priors,1)).astype(np.int64),np.zeros((num_priors, 16)).astype(np.float32)

    # [1,num_priors] best ground truth for each prior
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_idx_filter.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    best_truth_overlap.index_fill_(0, best_prior_idx_filter, 2)  # ensure best prior
    # TODO refactor: index  best_prior_idx with long tensor
    # ensure every gt matches with its prior of max overlap
    for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
    matches = truths[best_truth_idx]  # Shape: [num_priors,14]
    conf = labels[best_truth_idx]  # Shape: [num_priors]
    keypoints=keypoints[best_truth_idx]
    conf[best_truth_overlap < threshold] = 0  # label as background

    loc = encode(matches, priors2, variances)
    return loc, conf,keypoints



def encode(matched, priors, variances):
    """

    Args:
        matched (tensor): Coords of ground truth for each prior in xyxy Shape: [num_priors, 4].
        priors (tensor): Prior boxes in center-offset form Shape: [num_priors,4].
        variances (list[float]):  Variances of priorboxes

    Returns:
        encoded boxes and landmarks (tensor), Shape: [num_priors, 14]

    """

    # dist b/t match center and prior's center
    priors = priors.clone()
    g_cxcy = (matched[:, 0:2] + matched[:, 2:4]) / 2 - priors[:, 0:2]
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:4])
    # match wh / prior wh
    g_wh = (matched[:, 2:4] - matched[:, 0:2]) / priors[:, 2:4]
    g_wh = torch.log(g_wh) / variances[1]

    # # landmarks
    # g_xy1 = (matched[:, 4:6] - priors[:, 0:2]) / (variances[0] * priors[:, 2:4])
    # g_xy2 = (matched[:, 6:8] - priors[:, 0:2]) / (variances[0] * priors[:, 2:4])
    # g_xy3 = (matched[:, 8:10] - priors[:, 0:2]) / (variances[0] * priors[:, 2:4])
    # g_xy4 = (matched[:, 10:12] - priors[:, 0:2]) / (variances[0] * priors[:, 2:4])
    # g_xy5 = (matched[:, 12:14] - priors[:, 0:2]) / (variances[0] * priors[:, 2:4])

    # return target for loss
    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,14]


def decode(loc, priors, variances):
    """Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Adapted from https://github.com/Hakuyume/chainer-ssd

    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes

    Return:
        decoded bounding box predictions

    """
    boxes = torch.cat([priors.unsqueeze(0)[:, :, 0:2] + loc[:, :, 0:2] * variances[0] * priors.unsqueeze(0)[:, :, 2:4],
                       priors.unsqueeze(0)[:, :, 2:4] * torch.exp(loc[:, :, 2:4] * variances[1])], -1)
    boxes[:, :, 0:2] -= boxes[:, :, 2:4] / 2
    boxes[:, :, 2:4] += boxes[:, :, 0:2]
    return boxes



In [ ]:
np.set_printoptions(precision=3)

def point_in_box(point,box):
    if ndim(box)+1==ndim(point):
        box=np.expand_dims(box,1)
    if ndim(point)==ndim(box)==1:
        x,y=point
        x1,y1,x2,y2=box
        if x1<x<x2 and y1<y<y2:
            return True
        else:
            return False
    elif  ndim(point)==ndim(box)==2:
        x=point[:,0]
        y=point[:,1]
        x1=box[:,0]
        y1=box[:,1]
        x2=box[:,2]
        y2=box[:,3]
        return np.greater_equal(x,x1)*np.greater_equal(x2,x)*np.greater_equal(y,y1)*np.greater_equal(y2,y)
    elif  ndim(point)==ndim(box)==3:
        x=point[:,:,0]
        y=point[:,:,1]
        x1=box[:,:,0]
        y1=box[:,:,1]
        x2=box[:,:,2]
        y2=box[:,:,3]
        return np.greater_equal(x,x1)*np.greater_equal(x2,x)*np.greater_equal(y,y1)*np.greater_equal(y2,y)
   

In [ ]:

def xywh2xyxy(locations):
    return np.concatenate([locations[..., :2] - locations[..., 2:] / 2.0, locations[..., :2] + locations[..., 2:] / 2.0],
                     axis=1)

class SsdBboxDataset(BboxDataset):
    def __init__(self, boxes=None, image_size=(480, 640), priors=None, center_variance=0.1, size_variance=0.2,
                 gt_overlap_tolerance=0.5, object_type=ObjectType.absolute_bbox, class_names=None,
                 symbol='bbox', name=''):
        super().__init__(boxes=boxes, image_size=image_size, object_type=object_type, class_names=class_names,
                         symbol=symbol, name=name)
        self._element_spec = TensorSpec(shape=TensorShape([None, 21]), name=self.symbol, object_type=self.object_type, is_spatial=True)
        self.priors = priors
        self.image_size = image_size
        self.center_variance = center_variance
        self.size_variance = size_variance
        self.label_transform_funcs = []
        self.gt_overlap_tolerance = gt_overlap_tolerance
        self.bbox_post_transform_funcs = []
        print('priors',self.priors.shape,self.priors )

    def __getitem__(self, index: int):

        # 取出xyxy定義boxes或是[]
        # 為了避免[]清單與np.ndarray計算差異，因此補上np.array([[-1,-1,-1,-1,-1]])替代
        if self.items[index] is None or self.items[index] == []:
            return np.array([-1] * 21)
        else:
            
            return self.items[index].astype(np.float32)

    def area_of(self, left_top, right_bottom):
        """Compute the areas of rectangles given two corners.

        Args:
            left_top (N, 2): left top corner.
            right_bottom (N, 2): right bottom corner.

        Returns:
            area (N): return the area.
        """
        hw = np.clip(right_bottom - left_top, 0.0, None)
        return hw[..., 0] * hw[..., 1]

    def iou_of(self, boxes0, boxes1, eps=1e-5):
        """Return intersection-over-union (Jaccard index) of boxes.

        Args:
            boxes0 (N, 4): ground truth boxes.
            boxes1 (N or 1, 4): predicted boxes.
            eps: a small number to avoid 0 as denominator.
        Returns:
            iou (N): IoU values.
        """
        overlap_left_top = np.maximum(boxes0[..., :2], boxes1[..., :2])
        overlap_right_bottom = np.minimum(boxes0[..., 2:], boxes1[..., 2:])

        overlap_area = self.area_of(overlap_left_top, overlap_right_bottom)
        area0 = self.area_of(boxes0[..., :2], boxes0[..., 2:])
        area1 = self.area_of(boxes1[..., :2], boxes1[..., 2:])
        return overlap_area / (area0 + area1 - overlap_area + eps)

    def convert_boxes_to_locations(self, center_form_boxes, center_form_priors):
        if len(center_form_priors.shape) + 1 == len(center_form_boxes.shape):
            center_form_priors = np.expand_dims(center_form_priors, 0)
        return np.concatenate([(center_form_boxes[..., :2] - center_form_priors[..., :2]) / center_form_priors[...,
                                                                                            2:] / self.center_variance,
                               np.log(np.clip(center_form_boxes[..., 2:] / center_form_priors[..., 2:], 1e-8, np.inf)) / self.size_variance],
                              axis=len(center_form_boxes.shape) - 1)

    def assign_priors(self, gt_boxes, gt_labels,center_form_priors, iou_threshold):

        corner_form_priors = xywh2xyxy(center_form_priors)
        #print('corner_form_priors',corner_form_priors.shape,corner_form_priors)
        ious = self.iou_of(np.expand_dims(gt_boxes, 0), np.expand_dims(corner_form_priors, 1))
    
        # size: num_priors
        best_target_per_prior, best_target_per_prior_index = np.max(ious, axis=1), np.argmax(ious, axis=1)
    
        best_prior_per_target, best_prior_per_target_index = np.max(ious, axis=0), np.argmax(ious, axis=0)
    
        
        for target_index, prior_index in enumerate(best_prior_per_target_index):
            best_target_per_prior_index[prior_index] = target_index
        
        #2.0 is used to make sure every target has a prior assigned
        best_prior_per_target_index_list = best_prior_per_target_index.tolist()
        for i in range(best_target_per_prior.shape[0]):
            if i in best_prior_per_target_index_list:
                best_target_per_prior[i] = 2


       
        labels= gt_labels[best_target_per_prior_index]
        labels[best_target_per_prior<2]=0
    
        boxes = gt_boxes[best_target_per_prior_index]


        return boxes, labels

    def data_transform(self, data):
        if data is None or len(data) == 0:
            return np.zeros((self.priors.shape[0], 21)).astype(np.float32)
        elif isinstance(data, np.ndarray):
            height, width = self.image_size

            data[:, 0] = np.clip(data[:, 0], 0, width)
            data[:, 2] = np.clip(data[:, 2], 0, width)
            data[:, 1] = np.clip(data[:, 1], 0, height)
            data[:, 3] = np.clip(data[:, 3], 0, height)
            
            
            small_box_mask1=np.round(data[:, 0])==np.round(data[:, 2],0)
            data[:, 2:3][small_box_mask1,:]+=1
            small_box_mask2=np.round(data[:, 1])==np.round(data[:, 3],0)
            data[:, 3:4][small_box_mask2,:]+=1
            
            
            box_w = np.clip(np.expand_dims(data[:, 2] - data[:, 0], -1),1,640)
            box_h = np.clip(np.expand_dims(data[:, 3] - data[:, 1], -1),1,480)
            box_left = np.expand_dims(data[:, 0].copy(), -1)
            box_top = np.expand_dims(data[:, 1].copy(), -1)
            # print('box_w',box_w.shape)
            # print('box_left',box_left.shape)

            gt_box = data[:, :4]
            gt_label = data[:, 4:5]
            data[:, 4:5]=1
            gt_box[:, 0] = gt_box[:, 0] / float(width)
            gt_box[:, 2] = gt_box[:, 2] / float(width)
            gt_box[:, 1] = gt_box[:, 1] / float(height)
            gt_box[:, 3] = gt_box[:, 3] / float(height)

        

            if gt_box is not None and len(gt_box) > 0:
                truths = to_tensor(gt_box).float()
                labels = to_tensor(gt_label).long()
      
                
                boxes, confidences = self.assign_priors(gt_box, gt_label,  to_numpy(self.priors.copy()), 0.5)
                boxes = xyxy2xywh(boxes)
                locations = self.convert_boxes_to_locations(boxes, to_numpy(self.priors.copy()))

                locations = np.concatenate([to_numpy(locations).astype(np.float32),to_numpy(confidences).astype(np.float32)], axis=-1)
                return to_numpy(locations).astype(np.float32)

            num_priors = self.priors.shape[0]
            return np.zeros((num_priors, 5)).astype(np.float32)


    def bbox_transform(self, bbox):
        return self.data_transform(bbox)


In [ ]:
imgs=[]
boxes=[]
for k,v in bboxes_dataset_dict.items():
    imgs.append('../input/nfl-health-and-safety-helmet-assignment/images/'+k)
    new_shape=list(v.shape)
    new_shape[-1]=5
    new_v=np.ones(new_shape)
    new_v[:,:4]=v
    boxes.append(new_v)
print(boxes[0])
    
    

ds1=ImageDataset(imgs,object_type=ObjectType.rgb,symbol='images')
ds2=SsdBboxDataset(boxes,image_size=(480, 640),priors=rfbmodel.model.priors,symbol='boxes')

data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2))
#data_provider.batch_transform_funcs=[OneOf([ImageMosaic((480,640),keep_prob=0.4),DetectionMixup((480,640),keep_prob=0.4)])]
#data_provider.batch_transform_funcs=[ImageMosaic((480,640),keep_prob=0.6)]
data_provider.paired_transform_funcs=[
    Resize((600,800)),
    RandomTransform(rotation_range=25,zoom_range=0.1, shift_range=0.1,  shear_range= 0.4,random_flip= 0.2),
    RandomCrop((480,640),scale_range=(0.2,1.5))]
    #RandomGridMask(size_range=(480,640),max_d1=64,rotation_range=(10,30),keep_prob=0.6)]
#data_provider.image_transform_funcs=[]
#設定影像轉換
data_provider.image_transform_funcs=[
    RandomAdjustGamma(gamma_range=(0.5,1.5)),#調整明暗
    RandomAdjustHue(scale=(-0.3,0.3)),#調整色相
    RandomAdjustContrast(scale=(0.6,1.4)),#調整對比
    RandomAdjustSaturation(scale=(0.6,1.4)),#調整飽和度
    RandomErasing(size_range=(0.05, 0.1), transparency_range=(0.4, 0.8), transparancy_ratio=1, keep_prob=0.5), #加入隨機擦去
    RandomBlur(scale=(3,7)),#隨機模糊
    GrayMixRGB(keep_prob=0.75),
    SaltPepperNoise(prob=0.002),#椒鹽噪音
    RandomErasing(size_range=(0.05, 0.1), transparency_range=(0.4, 0.8), transparancy_ratio=0.5, keep_prob=0.5), #加入隨機擦去
    Normalize(127.5,127.5)] #標準化



In [ ]:
imgs,boxes=data_provider.next()

print(boxes.shape)
target_confidences=boxes[:,:,4]
target_boxes=boxes[:,:,:4]
this_target_confidences=target_confidences[target_confidences>0]

this_target_boxes=target_boxes[target_confidences>0]


print(len(this_target_confidences))
print(boxes[target_confidences>0])


In [ ]:
data_provider.preview_images()

In [ ]:
def get_actual_target(target_locations,target_confidences,priors,center_variance=0.1, size_variance=0.2):
    relative_target_boxes = decode(target_locations[:,:,:4].copy(),priors, (center_variance,size_variance))
    relative_target_boxes=to_numpy(relative_target_boxes)
    target_boxes = decode(target_locations[:,:,:4].copy(),priors, (center_variance,size_variance))*to_tensor([[640,480,640,480]])
    target_boxes=np.round(to_numpy(target_boxes),0)
    target_confidences=to_numpy(target_confidences)
    all_sorted_boxes=[]
    all_target_confidences=[]
    for i in range(len(target_boxes)):
        this_target_boxes=target_boxes[i][target_confidences[i]>0,:]
        this_relative_target_boxes=relative_target_boxes[i][target_confidences[i]>0,:]
        arr_idx=this_target_boxes[:,0]*1000000+this_target_boxes[:,1]*10000+this_target_boxes[:,2]*100+this_target_boxes[:,3]
        np.sort(arr_idx)
        sorted_indexes=np.argsort(arr_idx)
        
        sorted_target_confidences=target_confidences[i][target_confidences[i]>0][sorted_indexes]
        
        sorted_target_boxes=this_target_boxes[sorted_indexes]
        sorted_relative_target_boxes=this_relative_target_boxes[sorted_indexes]
        
        mask=np.ones(len(sorted_target_boxes))
        for n in range(len(sorted_target_boxes)):
            if n>0 and np.array_equal(sorted_target_boxes[n],sorted_target_boxes[n-1]):
                mask[n]=0
        mask=mask.astype(np.bool)
        sorted_target_confidences=sorted_target_confidences[mask]
        sorted_target_boxes=sorted_target_boxes[mask,:]
        sorted_relative_target_boxes=sorted_relative_target_boxes[mask,:]
        #print(sorted_target_confidences)
        label=np.zeros((priors.shape[0]))
        if len(sorted_target_boxes)>0:
            corner_form_priors = xywh2xyxy(to_numpy(priors)).astype(np.float32)
            
            ious = ds2.iou_of(np.expand_dims(sorted_relative_target_boxes.astype(np.float32), 0), np.expand_dims(corner_form_priors, 1))
            best_target_per_prior, best_target_per_prior_index = np.max(ious, axis=1), np.argmax(ious, axis=1)
            best_prior_per_target, best_prior_per_target_index = np.max(ious, axis=0), np.argmax(ious, axis=0)
            #print(best_prior_per_target_index)
            #print(sorted_target_confidences)
            
            for n,p in enumerate(best_prior_per_target_index):
                label[p]=sorted_target_confidences[n]
            #print(label)
            all_target_confidences.append(label )
            all_sorted_boxes.append(sorted_target_boxes)
        else:
            all_target_confidences.append(label)
            all_sorted_boxes.append(sorted_target_boxes)
        
        
    return all_sorted_boxes,all_target_confidences

In [ ]:
import torch.nn.functional as F
def preprocess_data(training_context):
    traindata=training_context['train_data']
    boxes=traindata['boxes']
    traindata['target_confidences']=boxes[:,:, 4].long().detach()
    traindata['target_locations']=boxes[:,:,:4].detach()

    
    
def hard_negative_mining(loss,labels, neg_pos_ratio):
    """
    It used to suppress the presence of a large number of negative prediction.
    It works on image level not batch level.
    For any example/image, it keeps all the positive predictions and
     cut the number of negative predictions to make sure the ratio
     between the negative examples and positive examples is no more
     the given ratio for an image.

    Args:
        loss (N, num_priors): the loss for each example.
        labels (N, num_priors): the labels.
        neg_pos_ratio:  the ratio between the negative examples and positive examples.
    """
    pos_mask = labels > 0
    #print('loss',loss.shape,loss)

    num_pos = pos_mask.long().sum(dim=-1, keepdim=True)
    #print('num_pos',num_pos.shape,num_pos)
    #print('num_pos',num_pos)
    #避免此圖片無正案例
    num_neg = clip(num_pos,min=1) * neg_pos_ratio
    #print('num_neg',num_neg)

    
    loss[pos_mask] = -math.inf
    _, indexes = loss.sort(dim=-1,descending=True)
    _, orders = indexes.sort(dim=-1)
    #print('orders',orders.shape)
    neg_mask = orders < num_neg

    return pos_mask | neg_mask


def jaccard(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.  Here we operate on
    ground truth boxes and default boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2] - box_a[:, 0]) * (box_a[:, 3] - box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2] - box_b[:, 0]) * (box_b[:, 3] - box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter,union 


def area_of(left_top, right_bottom):
    """Compute the areas of rectangles given two corners.

    Args:
        left_top (N, 2): left top corner.
        right_bottom (N, 2): right bottom corner.

    Returns:
        area (N): return the area.
    """
    hw = clip(right_bottom - left_top, 0.0, None)
    return hw[..., 0] * hw[..., 1]

def iou_of(boxes0, boxes1, eps=1e-5):
    """Return intersection-over-union (Jaccard index) of boxes.

    Args:
        boxes0 (N, 4): ground truth boxes.
        boxes1 (N or 1, 4): predicted boxes.
        eps: a small number to avoid 0 as denominator.
    Returns:
        iou (N): IoU values.
    """
    overlap_left_top = maximum(boxes0[..., :2], boxes1[..., :2])
    overlap_right_bottom = minimum(boxes0[..., 2:], boxes1[..., 2:])

    overlap_area =area_of(overlap_left_top, overlap_right_bottom)
    area0 = area_of(boxes0[..., :2], boxes0[..., 2:])
    area1 = area_of(boxes1[..., :2], boxes1[..., 2:])
    return overlap_area / (area0 + area1 - overlap_area + eps),overlap_area ,area0 + area1 - overlap_area

def hard_nms(box_scores, nms_threshold, top_k=-1, candidate_size=200):
    """

    Args:
        box_scores (N, 5): boxes in corner-form and probabilities.
        nms_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
        candidate_size: only consider the candidates with the highest scores.
    Returns:
         picked: a list of indexes of the kept boxes
    """
    if box_scores is None or len(box_scores) == 0:
        return None, None
    all_scores = box_scores[:, -1]
    all_class = box_scores[:, 4]
    all_boxes = box_scores[:, :4]
    all_index =np.arange(len(box_scores))
    all_picked = []
    for clas in [1,2,3]:
        scores=all_scores[all_class==clas]
        boxes=all_boxes[all_class==clas,:]
        this_index=all_index[all_class==clas]
        
        picked = []
        #_, indexes = scores.sort(descending=True)
        indexes = np.argsort(-scores)
        indexes = indexes[:candidate_size]
        #indexes = indexes[-candidate_size:]
        while len(indexes) > 0:
            # current = indexes[0]
            current = indexes[-1]
            picked.append(current)
            if 0 < top_k == len(picked) or len(indexes) == 1:
                break
            current_box = boxes[current, :]
            # indexes = indexes[1:]
            indexes = indexes[:-1]
            rest_boxes = boxes[indexes, :]
            iou, inter, union = iou_of(rest_boxes, expand_dims(current_box, axis=0), )
            iot = inter / area_of(current_box[..., :2], current_box[..., 2:])

            #indexes = indexes[((iou <= nms_threshold) * (iot <= nms_threshold)).astype(np.bool)]
            indexes = indexes[(iou <= nms_threshold).astype(np.bool)]
            all_picked.extend(this_index[picked])

    return box_scores[all_picked, :], all_picked

def soft_nms(box_scores, nms_threshold, top_k=-1, candidate_size=200, sigma=0.5):
    """

    Args:
        box_scores (N, 5): boxes in corner-form and probabilities.
        nms_threshold: intersection over union threshold.
        top_k: keep top_k results. If k <= 0, keep all the results.
        candidate_size: only consider the candidates with the highest scores.
    Returns:
         picked: a list of indexes of the kept boxes
    """
    picked_box_scores = []
    while box_scores.size(0) > 0:
        max_score_index = torch.argmax(box_scores[:, 4])
        cur_box_prob = torch.tensor(box_scores[max_score_index, :])
        picked_box_scores.append(cur_box_prob)
        if len(picked_box_scores) == top_k > 0 or box_scores.size(0) == 1:
            break
        cur_box = cur_box_prob[:-1]
        box_scores[max_score_index, :] = box_scores[-1, :]
        box_scores = box_scores[:-1, :]
        ious = iou_of(cur_box.unsqueeze(0), box_scores[:, :-1])
        box_scores[:, -1] = box_scores[:, -1] * torch.exp(-(ious * ious) / sigma)
        box_scores = box_scores[box_scores[:, -1] > score_threshold, :]
    if len(picked_box_scores) > 0:
        return torch.stack(picked_box_scores)
    else:
        return torch.tensor([])




class MultiBoxLoss(nn.Module):
    def __init__(self, priors, neg_pos_ratio=3.5, center_variance=0.1, size_variance=0.2):
        """Implement SSD Multibox Loss.

        Basically, Multibox loss combines classification loss
         and Smooth L1 regression loss.
        """
        super(MultiBoxLoss, self).__init__()
        self.neg_pos_ratio = neg_pos_ratio
        self.center_variance = center_variance
        self.size_variance = size_variance
        self.priors = to_tensor(priors)
        self.ce=CrossEntropyLoss(axis=-1,reduction='mean')

    def forward(self, confidences, locations, target_confidences, target_locations):
        """Compute classification loss and smooth l1 loss.

        Args:
            confidence (batch_size, num_priors, num_classes): class predictions.
            locations (batch_size, num_priors, 4): predicted locations.
            target_confidence (batch_size, num_priors): real labels of all the priors.
            target_locations (batch_size, num_priors, 4): real boxes corresponding all the priors.
        """
        
        num_classes = 2
        batch= confidences.size(0)
        boxes =locations[...,:4]
        
        
        target_boxes =target_locations[...,:4]
        #target_boxes = decode(target_boxes,self.priors, (self.center_variance,self.size_variance))#*to_tensor([[640,480,640,480]])
       
        regression_loss=0
        classification_loss=0
        for i in range(batch):
            this_target_confidences=target_confidences[i,:]
            this_confidences=confidences[i,:,:]
            this_boxes=boxes[i,:,:]
            this_target_boxes=target_boxes[i,:,:]
            
            pos_mask = this_target_confidences > 0
            masked_boxes =this_boxes[pos_mask, :]#.view((-1,4))
            masked_target_boxes =this_target_boxes[pos_mask, :].detach()
            
            pos_masked_confidences =this_confidences[pos_mask, :].reshape(-1, num_classes)
            pos_masked_target_confidences =this_target_confidences[pos_mask].reshape(-1).detach()
            if len(masked_target_boxes):
                #print('masked_boxes:',masked_boxes.shape,'masked_target_boxes:',masked_target_boxes.shape)
                #print('pos_masked_confidences:',pos_masked_confidences.shape,'pos_masked_target_confidences:',pos_masked_target_confidences.shape)
                regression_loss=regression_loss+8*F.smooth_l1_loss(masked_boxes, masked_target_boxes.detach())#+AdaptiveWingLoss()(masked_boxes, masked_target_boxes)
                #classification_loss=classification_loss+0.2*self.ce(pos_masked_confidences,pos_masked_target_confidences.detach())
                
        with torch.no_grad():
            #已經取log_softmax了不需要再做一次
            #loss=-confidences[:,:,0]#.sum(-1)
            loss = -this_confidences[:,0]
            mask = hard_negative_mining(loss, this_target_confidences, self.neg_pos_ratio)
        masked_confidences =this_confidences[mask, :]#.reshape(-1, num_classes)
        masked_target_confidences = this_target_confidences[mask]#.reshape(-1).detach()
        classification_loss = classification_loss+self.ce(masked_confidences,masked_target_confidences.detach())


           
        return (classification_loss+regression_loss)/batch
    

@torch.no_grad()
def classification_recall(confidences,target_locations,target_confidences):
    all_sorted_boxes,all_target_confidences=get_actual_target(target_locations,target_confidences,rfbmodel.model.priors,center_variance=0.1, size_variance=0.2)
    
    
    confidences=to_numpy(exp(confidences))
    target_confidences=np.stack(all_target_confidences,0)#to_numpy(target_confidences)
    mask=target_confidences>0
    #mask=((np.argmax(confidences[:,:,:],-1)>0)*(confidences[:,:,1:].max(-1)>=0.5)).astype(np.bool)
    #print('np.argmax(confidences[:,:,1:],-1)+1',(np.argmax(confidences[:,:,1:],-1)+1).shape)
    #print('confidences[:,:,0]<0.6',(confidences[:,:,0]<0.6).shape)
    

    pred_confidences=confidences[:,:,1]
    least_confidences=confidences.max(-1)
    
    
    pred_confidences[least_confidences>=0.5]=1
    
    masked_confidences=pred_confidences[mask]
    masked_target_confidences=target_confidences[mask]
    return equal(masked_confidences,masked_target_confidences).mean()

@torch.no_grad()
def classification_accuracy(confidences,target_locations,target_confidences):
    all_sorted_boxes,all_target_confidences=get_actual_target(target_locations,target_confidences,rfbmodel.model.priors,center_variance=0.1, size_variance=0.2)
    
    
    confidences=to_numpy(exp(confidences))
    target_confidences=to_numpy(target_confidences)
    #mask=np.argmax(confidences[:,:,:],-1)>0
    mask=np.argmax(confidences,-1)>0

    masked_confidences=np.argmax(confidences,-1)[mask]
    masked_target_confidences=target_confidences[mask]
    if len(masked_confidences)>0:
        return equal(masked_confidences,masked_target_confidences).mean()
    else:
        return 0


@torch.no_grad()
def bbox_rmse(locations,target_locations,target_confidences):
    bboxes=locations[:,:,:4].copy().detach()
    target_bboxes=target_locations[:,:,:4].copy().detach()
    bboxes = clip(decode(bboxes, rfbmodel.model.priors, (0.1,0.2)),min=0,max=1)
    target_bboxes = decode(target_bboxes, rfbmodel.model.priors, (0.1,0.2))
    
    
    
    mask=target_confidences>0
    masked_bboxes=bboxes[mask,:]
    masked_target_bboxes=target_bboxes[mask,:]
   
    if len(masked_target_bboxes)>0:
        return ((masked_bboxes-masked_target_bboxes)**2).mean().sqrt()
    else:
        return 0

    
    
    
def bbox_iou(bboxes1, bboxes2):
    """

    Args:
        bboxes1 (Tensor): shape (n, 4)
        bboxes2 (Tensor): shape (k, 4)

    Returns:
         ious(Tensor): shape (n, k)

    Examples;
    >>> boxes1=to_tensor(np.array([[39, 63, 203, 112], [49, 75, 203, 125],[31, 69, 201, 125],[50, 72, 197, 121],[35, 51, 196, 110]]))
    >>> boxes2=to_tensor(np.array([[54, 66, 198, 114], [42, 78, 186, 126], [18, 63, 235, 135],[54, 72, 198, 120],[36, 60, 180, 108]]))
    >>> iou_loss=(1-bbox_iou(boxes1,boxes2)).sum()/(boxes1.shape[0]*boxes2.shape[0])
    >>> print(iou_loss.cpu())
    tensor(0.3802)

    >>> boxes1=to_tensor(np.array([[39, 63, 203, 112], [49, 75, 203, 125],[31, 69, 201, 125],[50, 72, 197, 121],[35, 51, 196, 110]]))
    >>> boxes2=to_tensor(np.array([[54, 66, 198, 114], [42, 78, 186, 126], [18, 63, 235, 135],[54, 72, 198, 120]]))
    >>> iou_loss=(1-bbox_iou(boxes1,boxes2)).sum()/(boxes1.shape[0]*boxes2.shape[0])
    >>> print(iou_loss.cpu())
    tensor(0.3703)




    """
    rows = bboxes1.shape[0]
    cols = bboxes2.shape[0]
    ious = torch.zeros((rows, cols))
    if rows * cols == 0:
        return ious
    exchange = False
    if bboxes1.shape[0] > bboxes2.shape[0]:
        bboxes1, bboxes2 = bboxes2, bboxes1
        ious = torch.zeros((cols, rows))
        exchange = True
    area1 = (bboxes1[:, 2] - bboxes1[:, 0]) * (bboxes1[:, 3] - bboxes1[:, 1])
    area2 = (bboxes2[:, 2] - bboxes2[:, 0]) * (bboxes2[:, 3] - bboxes2[:, 1])

    lt = maximum(bboxes1[:, None, :2], bboxes2[:, :2])  # [N,M,2]
    rb = minimum(bboxes1[:, None, 2:], bboxes2[:, 2:])  # [N,M,2]
    wh = torch.clamp(rb - lt, min=0)  # [N,M,2]
    inter_area = wh[:, :, 0] * wh[:, :, 1]  # [N,M]
    #print('inter_area',inter_area)


    union =  area1[:, None] + area2 -inter_area
    #print('union',union)
    ious = inter_area / union
    ious = torch.clamp(ious,min=0,max = 1.0)
    if exchange:
        ious = ious.T
    #print('ious',ious)
    return reduce_max(ious,0).mean()

@torch.no_grad()
def iou(confidences, locations, target_confidences, target_locations):
    iou=0
    confidences=to_numpy(exp(confidences))
    target_confidences=to_numpy(target_confidences)
    boxes1 =locations[:,:,:4]
    boxes1 = clip(to_numpy(decode(boxes1, rfbmodel.model.priors, (0.1,0.2))*to_tensor([[640,480,640,480]])),min=0)

    
    boxes2 =target_locations[:,:,:4]
    boxes2 =  to_numpy(decode(boxes2, rfbmodel.model.priors, (0.1,0.2))*to_tensor([[640,480,640,480]]))
    all_sorted_boxes,all_target_confidences=get_actual_target(target_locations[:,:,:4],target_confidences,rfbmodel.model.priors,center_variance=0.1, size_variance=0.2)
    
    num_batch=0
    for i in range(len(locations)):
        #mask1=((np.argmax(confidences[i][:,:],-1)>0)*(confidences[i][:,1:].max()>0.5)).astype(np.bool)
        mask1=confidences[i][:,1]>0.35
        mask2=target_confidences[i,:]>0
        #預測框遮罩包括應有框位置(mask2)以及預測出框(mask1)兩者的聯集
   
        this_boxes1=boxes1[i,:,:][mask1,:]
     
        this_boxes1_conf=np.expand_dims(np.argmax(confidences[i],-1)[mask1],-1)
        this_boxes1_score=np.expand_dims(np.max(confidences[i],-1)[mask1],-1)
        box_probs = np.concatenate([this_boxes1.astype(np.float32),this_boxes1_conf.astype(np.float32),this_boxes1_score.astype(np.float32)], axis=1)
        
        if len(box_probs) > 1:
            box_probs,keep = hard_nms(box_probs, 0.5, top_k=-1, )
            this_boxes1=this_boxes1[keep,:]
            this_boxes1_conf=this_boxes1_conf[keep,:]
            #print(this_boxes1)
    
    
        #this_boxes2=boxes2[i,:,:][mask2,:]
        this_boxes2=all_sorted_boxes[i]
        #print(this_boxes2)
        
        if this_boxes2 is not None and len(this_boxes1)>0 and len(this_boxes2)>0:
            iou_results=to_numpy(bbox_iou(to_tensor(this_boxes1),to_tensor(this_boxes2)))
            #print(iou_results.mean())
            iou=iou+iou_results
            num_batch+=1
        elif  len(this_boxes2)>0:
            num_batch+=1
    return iou/num_batch

In [ ]:
from PIL import Image, ImageDraw

def draw_detection(training_context):

    traindata=training_context['train_data']
    #print(traindata.keys())
    imagedata=traindata['images'].copy()
    labeldata=traindata['locations'].copy()
    #print(softmax(traindata['confidences'].float(),-1))
    #基於訓練階段輸出繪圖，信心水準應該要取exp
    confidences =to_numpy(exp(traindata['confidences'].copy().float()))

    boxes =to_tensor(labeldata[:,:,:4])

    gc.collect()
    #基於訓練階段輸出繪圖，boxes應該要decode
    locations = decode(boxes, rfbmodel.model.priors, (0.1,0.2))*to_tensor([[640,480,640,480]])
    for i in range(6):
        pillow_img=array2image(reverse_image_backend_adaption(imagedata[i]*127.5+127.5))
        draw = ImageDraw.Draw(pillow_img)
        mask=confidences[i][:,1]>0.35

        this_locations = to_numpy(locations[i])[mask, :]
        this_locations=np.stack([np.clip(this_locations[:,0],0,640),np.clip(this_locations[:,1],0,480),np.clip(this_locations[:,2],0,640),np.clip(this_locations[:,3],0,480)],axis=-1)
        this_confidences=confidences[i][mask,1]
   
        
        #print(this_locations.shape)
        if len(this_locations)>0:
            print('this_locations',this_locations.shape)
            print('this_confidences',this_confidences.shape)
            print(reduce_max(confidences[i][:,1]))
            
#             area = area_of(this_locations[..., :2], this_locations[..., 2:])
#             area_mask=area>=1
#             this_locations=this_locations[area_mask,:]
#             this_confidences=this_confidences[area_mask,:]
    

            if len(this_confidences)>0:
                
                
                print('this_locations',this_locations.shape,this_locations)
                print('this_confidences',this_confidences.shape,this_confidences)
                box_probs=np.concatenate([this_locations.astype(np.float32),np.expand_dims(this_confidences,-1).astype(np.float32)], axis=-1)
                #print(box_probs.shape)
                if len(boxes) > 1:
                    box_probs, keep = hard_nms(box_probs, 0.6, top_k=-1, )
                    this_locations=this_locations[keep,:]

                    this_confidences=this_confidences[keep]
         
                if len(this_locations)<=200:
    
                    for box,conf in zip(this_locations,this_confidences):
                        #print('box',[int(v) for v in box],'conf',['{0:.3f}'.format(v) for v in conf])
                        #print(box)

                        pillow_img =plot_bbox([int(v) for v in box], pillow_img, palette[2] ,'Helmet', line_thickness=2)
                        
                else:
                    print(this_confidences.shape)
                    print(this_confidences[:10])
                    print(reduce_max(confidences[i][:,1]))
                    print(len(this_locations))
                display.display(pillow_img)
       
        else:
            print(confidences[i].shape)
            print(reduce_max(confidences[i][:,1]))
            display.display(pillow_img)
            #pillow_img.save('data_provider_next_{0}.jpg'.format(get_time_suffix()))

In [ ]:
is_resume=True
if is_resume and os.path.exists('./Models/nfl_ssd.pth'):
    rfbmodel.load_model('./Models/nfl_ssd.pth')
    print('./Models/nfl_ssd.pth loaded')
else:
    if os.path.exists('../input/nfl-health-safety-model/Models/nfl_ssd.pth'):
        rfbmodel.load_model('../input/nfl-health-safety-model/Models/nfl_ssd.pth')
        print('../input/nfl-health-safety-model/Models/nfl_ssd.pthh loaded')
        
        
rfbmodel.with_optimizer(optimizer=AdaBelief,lr=1e-3,betas=(0.9, 0.999))\
.with_loss(MultiBoxLoss(rfbmodel.model.priors,neg_pos_ratio=2.5))\
.with_metric(classification_accuracy, name='accuracy',print_only=True)\
.with_metric(classification_recall, name='recall',print_only=True)\
.with_metric(bbox_rmse,print_only=True)\
.with_metric(iou)\
.with_regularizer('l2',reg_weight=1e-6)\
.trigger_when(when='on_loss_calculation_start',frequency=1,unit='batch',action=preprocess_data)\
.trigger_when(when='on_batch_end',frequency=50,unit='batch',action=draw_detection)\
.with_model_save_path('./Models/nfl_ssd.pth')\
.with_learning_rate_scheduler(PolyLR(max_lr=1e-3,max_iter=3000))\
.with_automatic_mixed_precision_training()\

In [ ]:
plan=TrainingPlan()\
    .add_training_item(rfbmodel,name='rfbnet')\
    .with_data_loader(data_provider)\
    .with_batch_size(32)\
    .repeat_epochs(10)\
    .print_progress_scheduling(10,unit='batch')\
    .display_loss_metric_curve_scheduling(200)\
    .print_gradients_scheduling(100) \
    .save_model_scheduling(20,unit='batch')\
    .start_now()